In [0]:
%sql
use `na-dbxtraining`.biju_raw

In [0]:
%sql
-- Create the source Delta table
CREATE TABLE `na-dbxtraining`.biju_raw.tickets (
  ticket_id STRING NOT NULL,
  customer_id STRING,
  issue STRING,
  category STRING,
  resolution STRING,
  created_date DATE,
  resolved_date DATE,
  satisfaction_score INT,
  status STRING
) 
USING DELTA
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
);

-- IMPORTANT: Enable Change Data Feed for Delta Sync
-- This allows the vector index to automatically sync with table changes

-- Insert sample data
INSERT INTO `na-dbxtraining`.biju_raw.tickets VALUES
  ('TKT-1001', 'CUST001', 'Slow internet speeds in the evening', 'Network Performance', 
   'Identified network congestion. Upgraded customer to premium tier.', 
   '2024-10-10', '2024-10-15', 4, 'resolved'),
  ('TKT-1002', 'CUST002', 'Unable to connect to 5G network on new device', 'Technical Support',
   'APN settings were incorrect. Provided correct configuration.',
   '2024-10-18', '2024-10-20', 5, 'resolved'),
  ('TKT-1003', 'CUST003', 'Experiencing slow speeds after reaching data limit', 'Network Performance',
   'Customer was on throttled plan. Explained policy and offered upgrade.',
   '2024-10-22', '2024-10-25', 3, 'resolved');

-- Verify table creation
SELECT * FROM `na-dbxtraining`.biju_raw.tickets;

In [0]:
# Install the required package
%pip install databricks-vectorsearch

# Restart Python to ensure the package is available
dbutils.library.restartPython()

In [0]:
from databricks.vector_search.client import VectorSearchClient

# Initialize client
vsc = VectorSearchClient()

# Create endpoint
endpoint_name = "customer_support_endpoint"

try:
    vsc.create_endpoint(
        name=endpoint_name,
        endpoint_type="STANDARD"  # or "STORAGE_OPTIMIZED"
    )
    print(f"✓ Endpoint '{endpoint_name}' created successfully")
except Exception as e:
    print(f"Endpoint may already exist: {e}")

print(f"✓ Endpoint '{endpoint_name}' creation attempted")

after vector search index setup

In [0]:
from databricks.vector_search.client import VectorSearchClient

vsc = VectorSearchClient()

# Use the correct index name format: catalog_schema_table
index = vsc.get_index(
    endpoint_name="customer_support_endpoint",
    index_name="na-dbxtraining.biju_raw.ticket_embeddingsindex"
)

results = index.similarity_search(
    query_text="My internet is slow",
    columns=["ticket_id", "issue", "resolution"],
    num_results=3
)

print("🔍 Search Results:")
for result in results.get('result', {}).get('data_array', []):
    print(f"\n  Ticket: {result[0]}")
    print(f"  Issue: {result[1]}")
    print(f"  Resolution: {result[2]}")

The MCP Server URL is Now Ready!
Your Vector Search MCP server URL is:   https://adb-395997151148170.10.azuredatabricks.net/api/2.0/mcp/vector-search/prod/customer_support

Step 8: Connect to the MCP Server

In [0]:
%pip install databricks-mcp

In [0]:
%restart_python 

In [0]:
from databricks.sdk import WorkspaceClient
from databricks_mcp import DatabricksMCPClient
from databricks_mcp.oauth_provider import DatabricksOAuthClientProvider
from mcp.client.streamable_http import streamablehttp_client
from mcp.client.session import ClientSession

# Initialize workspace client
workspace_client = WorkspaceClient()
host = workspace_client.config.host

# Construct MCP server URL
mcp_server_url = f"{host}/api/2.0/mcp/vector-search/prod/customer_support"

# Connect to MCP server
async def connect_and_list_tools():
    async with streamablehttp_client(
        url=mcp_server_url,
        auth=DatabricksOAuthClientProvider(workspace_client)
    ) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            
            # List available tools
            response = await session.list_tools()
            print("🔧 Available MCP Tools:")
            for tool in response.tools:
                print(f"  - {tool.name}: {tool.description}")

import asyncio

if __name__ == "__main__":
    try:
        loop = asyncio.get_event_loop()
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    if loop.is_running():
        # If the loop is already running, create a task
        task = loop.create_task(connect_and_list_tools())
    else:
        loop.run_until_complete(connect_and_list_tools())

In [0]:
from databricks.sdk import WorkspaceClient
from databricks_mcp import DatabricksMCPClient

# Initialize
workspace_client = WorkspaceClient()
host = workspace_client.config.host

# Define MCP servers
MANAGED_MCP_SERVER_URLS = [
    f"{host}/api/2.0/mcp/vector-search/prod/customer_support",
    f"{host}/api/2.0/mcp/functions/prod/billing",
]

# Use in your agent code
# The agent can now call tools like:
# - ticket_embeddings.search(query="slow internet", num_results=5)

creating MCP agent

In [0]:
"""
Simple Customer Support Agent using Vector Search MCP Server
Run this in a Databricks notebook after your Vector Search MCP is set up
"""

from databricks.sdk import WorkspaceClient
from databricks_mcp import DatabricksMCPClient
import asyncio

# Your MCP server URL
VECTOR_SEARCH_MCP = "https://adb-1952652121322753.13.azuredatabricks.net/api/2.0/mcp/vector-search/na-dbxtraining/biju_raw"

# =============================================================================
# SIMPLE SYNCHRONOUS VERSION (Easy to understand)
# =============================================================================

def simple_support_agent(customer_issue: str):
    """
    Simple agent that searches for similar past tickets
    """
    from databricks.vector_search.client import VectorSearchClient
    
    # Initialize client
    vsc = VectorSearchClient(disable_notice=True)
    
    # Get the index
    index = vsc.get_index(
        endpoint_name="customer_support_endpoint",
        index_name="na-dbxtraining.biju_raw.ticket_embeddingsindex"
    )
    
    # Search for similar tickets
    results = index.similarity_search(
        query_text=customer_issue,
        columns=["ticket_id", "issue", "resolution"],
        num_results=3
    )
    
    # Format response
    print(f"\n{'='*80}")
    print(f"CUSTOMER ISSUE: {customer_issue}")
    print(f"{'='*80}\n")
    
    print("🔍 Found similar past tickets:\n")
    
    if 'result' in results and 'data_array' in results['result']:
        for i, result in enumerate(results['result']['data_array'], 1):
            ticket_id = result[0]
            issue = result[1]
            resolution = result[2]
            satisfaction = result[3]
            
            print(f"{i}. Ticket {ticket_id} (⭐ {satisfaction}/5)")
            print(f"   Issue: {issue}")
            print(f"   Resolution: {resolution}\n")
    else:
        print("No similar tickets found.")
    
    print(f"{'='*80}\n")

# =============================================================================
# TEST THE SIMPLE AGENT
# =============================================================================

# Example 1: Network issue
simple_support_agent("My internet is very slow during evening hours")

# Example 2: Technical issue
simple_support_agent("Can't connect to 5G on my new phone")

# Example 3: Billing issue
simple_support_agent("I have questions about my bill")


# =============================================================================
# ADVANCED: Using MCP Client (Proper way for production)
# =============================================================================

async def advanced_support_agent(customer_issue: str):
    """
    Advanced agent using MCP client
    This is the proper way to use MCP servers
    """
    from databricks_mcp.oauth_provider import DatabricksOAuthClientProvider
    from mcp.client.streamable_http import streamablehttp_client
    from mcp.client.session import ClientSession
    from mcp.types import CallToolRequest
    
    # Initialize workspace client
    workspace_client = WorkspaceClient()
    
    print(f"\n{'='*80}")
    print(f"CUSTOMER ISSUE: {customer_issue}")
    print(f"{'='*80}\n")
    
    # Connect to MCP server
    async with streamablehttp_client(
        url=VECTOR_SEARCH_MCP,
        auth=DatabricksOAuthClientProvider(workspace_client)
    ) as (read_stream, write_stream, _):
        
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            
            # List available tools
            tools_response = await session.list_tools()
            print(f"📋 Available MCP tools: {[t.name for t in tools_response.tools]}\n")
            
            # Call the vector search tool
            result = await session.call_tool(
                name="ticket_embeddings",
                arguments={
                    "query": customer_issue,
                    "num_results": 3
                }
            )
            
            print("🔍 MCP Tool Response:")
            print(result.content[0].text)
    
    print(f"\n{'='*80}\n")

# Test advanced agent
# asyncio.run(advanced_support_agent("My internet is very slow"))


# =============================================================================
# FULL AGENT WITH LLM (Production-ready)
# =============================================================================

def full_support_agent(customer_id: str, customer_issue: str):
    """
    Complete agent that:
    1. Searches for similar tickets (Vector Search MCP)
    2. Gets customer info (from tables)
    3. Recommends actions (using an LLM)
    """
    from databricks.vector_search.client import VectorSearchClient
    
    print(f"\n{'='*80}")
    print(f"CUSTOMER SUPPORT REQUEST")
    print(f"{'='*80}")
    print(f"Customer ID: {customer_id}")
    print(f"Issue: {customer_issue}\n")
    
    # Step 1: Get customer info
    print("📊 Step 1: Retrieving customer information...")
    customer_info = spark.sql(f"""
        SELECT * FROM `na-dbxtraining`.biju_raw.customers
        WHERE customer_id = '{customer_id}'
    """).collect()
    
    if customer_info:
        customer = customer_info[0].asDict()
        print(f"✓ Customer: {customer.get('name', 'Unknown')}")
        print(f"  Plan: {customer.get('plan', 'Unknown')}")
        print(f"  Data Usage: {customer.get('data_usage_gb', 0)} GB\n")
    else:
        print("⚠️ Customer not found\n")
        customer = {}
    
    # Step 2: Search for similar tickets using Vector Search MCP
    print("🔍 Step 2: Searching for similar past tickets...")
    vsc = VectorSearchClient(disable_notice=True)
    index = vsc.get_index(
        endpoint_name="customer_support_endpoint",
        index_name="na-dbxtraining.biju_raw.ticket_embeddingsindex"
    )
    
    results = index.similarity_search(
        query_text=customer_issue,
        columns=["ticket_id", "issue", "resolution"],
        num_results=3
    )
    
    if 'result' in results and 'data_array' in results['result']:
        print(f"✓ Found {len(results['result']['data_array'])} similar tickets:\n")
        for i, result in enumerate(results['result']['data_array'], 1):
            print(f"  {i}. {result[0]}: {result[1]}")
            print(f"     Resolution: {result[2]} (⭐ {result[3]}/5)\n")
    
    # Step 3: Generate recommendations
    print("💡 Step 3: Recommended Actions:")
    if results.get('result', {}).get('data_array'):
        best_match = results['result']['data_array'][0]
        print(f"  1. Apply proven solution from {best_match[0]}")
        print(f"     {best_match[2]}")
        print(f"  2. Follow up in 24 hours to confirm resolution")
        print(f"  3. Document the outcome for future reference")
    
    print(f"\n{'='*80}\n")

# =============================================================================
# USAGE EXAMPLES
# =============================================================================

print("\n" + "="*80)
print("TESTING CUSTOMER SUPPORT AGENT")
print("="*80 + "\n")

# First, create some test customer data if it doesn't exist
try:
    spark.sql("""
    CREATE TABLE IF NOT EXISTS `na-dbxtraining`.biju_raw.customers (
      customer_id STRING,
      name STRING,
      plan STRING,
      data_usage_gb DOUBLE
    ) USING DELTA
    """)
    
    spark.sql("""
    INSERT OVERWRITE `na-dbxtraining`.biju_raw.customers VALUES
      ('CUST001', 'John Smith', 'Premium Unlimited', 45.2),
      ('CUST002', 'Sarah Johnson', 'Basic 5GB', 4.8),
      ('CUST003', 'Mike Davis', 'Premium Unlimited', 78.5)
    """)
    print("✓ Test customer data ready\n")
except:
    pass

# Test scenarios
print("\n📞 SCENARIO 1: Network Performance Issue")
print("-" * 80)
full_support_agent("CUST003", "My internet is extremely slow during peak hours")

print("\n📞 SCENARIO 2: Technical Support Issue")
print("-" * 80)
full_support_agent("CUST002", "I can't connect to 5G network on my new device")

print("\n📞 SCENARIO 3: General Network Issue")
print("-" * 80)
full_support_agent("CUST001", "Internet keeps disconnecting")

print("\n" + "="*80)
print("✅ ALL TESTS COMPLETE!")
print("="*80)

In [0]:
# Create functions in SQL
spark.sql("""
CREATE FUNCTION prod.customer_support.calculate_credit(
  monthly_charge DOUBLE,
  days_remaining INT
)
RETURNS DOUBLE
RETURN monthly_charge * (days_remaining / 30.0)
""")

# MCP URL: 
# VECTOR_SEARCH_MCP = "https://adb-1952652121322753.13.azuredatabricks.net/api/2.0/mcp/vector-search/na-dbxtraining/biju_raw"

In [0]:
from databricks.vector_search.client import VectorSearchClient

# Test direct access (no MCP)
vsc = VectorSearchClient(disable_notice=True)
index = vsc.get_index(
    endpoint_name="customer_support_endpoint",
    index_name="na-dbxtraining.biju_raw.ticket_embeddingsindex"
)

# Test search
results = index.similarity_search(
    query_text="slow internet",
    columns=["ticket_id", "issue", "resolution"],
    num_results=3
)

print("✅ Vector Search works!")
for result in results['result']['data_array']:
    print(f"- {result[0]}: {result[1]}")

In [0]:
%sql
-- Verify table creation
SELECT * FROM `na-dbxtraining`.biju_raw.customers;